In [5]:
from skimage.feature import hog
from skimage import io,color
from skimage.transform import resize
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn import metrics
import os
from  LBP_descriptor import LocalBinaryPatterns
import commonfunctions as cf
import cv2
import pandas as pd
import csv
from skimage.feature import greycomatrix, greycoprops

# Get our training data 
X_train: features of training data.\
Y_train: labels of training data (1-->F, 0--> M).


In [6]:

# ICDAR LABELS 
labels_ICDAR=[]
with open("our dataset/train_answers.csv", 'r') as file:
    csvreader = csv.reader(file, delimiter=',')
    rows= np.array(list(csvreader))[1:].astype(float).astype(int)
for row in rows:
    labels_ICDAR.append(row[1])
    
def get_label_ICDAR(img):
    if img[0]=='0':
        if img[1]=='0': 
            return labels_ICDAR[int(img[2])-1]
        else: 
            return labels_ICDAR[int(img[1:3])-1]
    else: 
         return labels_ICDAR[int(img[0:3])-1]
        


def read_labels(path): 
    y=[]
    files = os.listdir(path)

    for file in files:
        if file[0]=='F':
            y.append(0)
        elif file[0]=='M':
            y.append(1)
        else: 
            y.append(get_label_ICDAR(file[1:4]))
          
    y=np.array(y).astype(float)
    return y 

Y_train= read_labels("Training_data/")
Y_test= read_labels("Test_data/")

# GLCM feature

In [7]:
def GLCM (image):

    # convert image to gray
    image= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    featureVector=[]
    
    # distances -> skipped pixels
    glcm = greycomatrix(image, distances=[5], angles=[45], levels=256,
                            symmetric=True, normed=True)
    featureVector.append(greycoprops(glcm, 'dissimilarity')[0, 0])
    featureVector.append(greycoprops(glcm, 'correlation')[0, 0])
    featureVector.append(greycoprops(glcm, 'contrast')[0, 0])
    featureVector.append(greycoprops(glcm, 'homogeneity')[0, 0])
    return featureVector

#--------------- LBP -----------    
desc = LocalBinaryPatterns(24, 8)

In [23]:
# TRY GLCM AND LBP ONLY
LBP_train=[]
GLCM_train=[]
X_train=[]
 
files = os.listdir("Training_data/")
for file in files:
    # read the image
    img = cv2.imread("Training_data/"+file )
    # ------------------- GLCM feature------------------------
    GLCM_train=GLCM(img)
    
    #------------------- LBP feature------------------------
    img2 = io.imread("Training_data/"+file )
    img2 = cf.downSize(img2 , 0.5)
    hist = desc.describe(img2)
    
    #--------------------------------------------------------
    # concatenate all the features in X_train   
    feature_vector_temp=np.hstack((GLCM_train,hist)).tolist()
    X_train.append(feature_vector_temp)
 
   

In [24]:
#TEST GLCM
LBP_test=[]
GLCM_test=[]
X_test=[]

files = os.listdir("Test_data/")

for file in files:
   
    # read te img
    img = cv2.imread("Test_data/" +file )
        
    #------------------- GLCM feature------------------------
    GLCM_test= GLCM(img)

     # #------------------- LBP feature------------------------
    img = io.imread("Test_data/" +file )
    img = cf.downSize(img , 0.5)
    hist = desc.describe(img)

    # concatenate all the features in X_train
    feature_test_temp=(np.hstack((GLCM_test,hist))).tolist()
    X_test.append( feature_test_temp )
  
X_test=np.array(X_test)

# Classification:

# 1. Random forest classifier

In [79]:

clf=RandomForestClassifier(n_estimators=1100)
clf.fit(X_train,Y_train)
Y_Predicted=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_Predicted)*100,"%")


Accuracy: 65.38461538461539 %


# 2. Linear SVM classifier

In [66]:
clf=LinearSVC(C=50.0, random_state=42)
clf.fit(X_train,Y_train )
Y_Predicted=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_Predicted)*100,"%")

Accuracy: 70.51282051282051 %


c:\Users\dell\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


# 3. Adaboost classifier

In [80]:
clf=AdaBoostClassifier(n_estimators=370)
clf.fit(X_train,Y_train)
Y_Predicted=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_Predicted)*100,"%")

Accuracy: 74.35897435897436 %


# 4. KNN classifier

In [60]:
accuracies=[]
for k in range(1,300):
    clf=KNeighborsClassifier(n_neighbors = k)
    clf.fit(X_train,Y_train)
    Y_Predicted=clf.predict(X_test)
    accuracies.append(metrics.accuracy_score(Y_test, Y_Predicted)*100)

print("Accuracy:",accuracies[np.argmax(accuracies)],"%")

Accuracy: 71.7948717948718 %
